In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Câu 1**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import export_text
from sklearn.tree import export_graphviz
import graphviz

# Đọc dữ liệu từ file data.xls
data = pd.read_excel("/content/drive/MyDrive/KPDL/TH6/input/Collected_Hr_data_performances.xls")

# Chọn các cột đặc trưng và cột nhãn
features = data[["Age", "Gender", "MaritalStatus", "EducationLevel", "EducationBackground",
                 "JobRole", "EnvironmentSatisfaction", "RelationshipSatisfaction",
                 "WorkLifeBalance", "TotalWorkExperienceInYears", "ExperienceYearsInCurrentRole"]]

labels = data["PerformanceResult"]

# Chuyển đổi các cột dữ liệu dạng văn bản thành dạng số (phải thực hiện để huấn luyện mô hình)
features = pd.get_dummies(features)
features

# Thay thế các ký tự không hợp lệ hoặc đặc biệt bằng khoảng trắng
features.columns = features.columns.str.replace('[^a-zA-Z0-9]', ' ', regex=True)

# Tách dữ liệu thành hai tập Train và Test
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.2, random_state=42)

# Xây dựng mô hình Cây quyết định (J48) và định nghĩa các giá trị tham số cần thử nghiệm
param_grid = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
# Tạo mô hình Cây quyết định (J48)
clf = DecisionTreeClassifier()

# Sử dụng GridSearchCV để thử nghiệm các tham số và lựa chọn mô hình tốt nhất
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')
grid_search.fit(train_features, train_labels)

# Lựa chọn mô hình tốt nhất sau khi thử nghiệm
best_clf = grid_search.best_estimator_

# In ra các thông số tốt nhất của mô hình
print("Best Parameters:", grid_search.best_params_)
print("Best Accuracy:", grid_search.best_score_)

# In các luật được học từ cây quyết định
tree_rules = export_text(best_clf, feature_names=list(train_features.columns))
print("Các luật đánh giá năng lực nhân viên:")
print(tree_rules)
print(train_features)
# Tạo biểu diễn đồ thị cho cây quyết định
dot_data = export_graphviz(best_clf, out_file=None, feature_names=list(train_features.columns),
                           class_names=best_clf.classes_, filled=True, rounded=True,
                           special_characters=True)

# Hiển thị đồ thị
graph = graphviz.Source(dot_data)
graph.render(filename='/content/drive/MyDrive/KPDL/TH6/output/Res/ds01', format='png', cleanup=True)
graph.view()

Best Parameters: {'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 2, 'splitter': 'random'}
Best Accuracy: 0.7364583333333333
Các luật đánh giá năng lực nhân viên:
|--- EnvironmentSatisfaction <= 1.47
|   |--- JobRole Developer <= 0.98
|   |   |--- RelationshipSatisfaction <= 3.72
|   |   |   |--- Age Main labor group <= 0.08
|   |   |   |   |--- MaritalStatus Single <= 0.62
|   |   |   |   |   |--- EducationBackground Marketing <= 0.29
|   |   |   |   |   |   |--- TotalWorkExperienceInYears <= 21.41
|   |   |   |   |   |   |   |--- RelationshipSatisfaction <= 2.67
|   |   |   |   |   |   |   |   |--- class: Meet Expectation
|   |   |   |   |   |   |   |--- RelationshipSatisfaction >  2.67
|   |   |   |   |   |   |   |   |--- class: Needs Improvement
|   |   |   |   |   |   |--- TotalWorkExperienceInYears >  21.41
|   |   |   |   |   |   |   |--- class: Needs Improvement
|   |   |   |   |   |--- EducationBackground Marketing >  0.29
|   |   |   |   |

'/content/drive/MyDrive/KPDL/TH6/output/Res/ds01.pdf'

In [ ]:
# Dự đoán kết quả trên tập Test bằng mô hình tốt nhất đã lựa chọn
predictions = best_clf.predict(test_features)

# Đánh giá hiệu suất của mô hình tốt nhất trên tập Test
accuracy = accuracy_score(test_labels, predictions)
print("Accuracy on Test Set:", accuracy)

Accuracy on Test Set: 0.7166666666666667


In [ ]:
# Tách dữ liệu thành hai tập Train và Test
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.2, random_state=42)

# Xây dựng mô hình Cây quyết định (REP Tree) với giới hạn độ sâu max_depth
max_depth = 10  # Độ sâu tùy chọn, bạn có thể điều chỉnh giá trị này
clf = DecisionTreeClassifier(max_depth=max_depth)

# Huấn luyện mô hình trên tập Train
clf.fit(train_features, train_labels)

# Dự đoán kết quả trên tập Test
predictions = clf.predict(test_features)

# Đánh giá hiệu suất của mô hình trên tập Test
accuracy = accuracy_score(test_labels, predictions)
print("Accuracy on Test Set:", accuracy)

Accuracy on Test Set: 0.7125


# **Câu 2**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.metrics import accuracy_score

# Đọc dữ liệu từ file 1-TNTHPT-XepLoai.csv
data = pd.read_csv("/content/drive/MyDrive/KPDL/TH6/input/1-TNTHPT-XepLoai.csv")

# Chọn các cột đặc trưng (điểm từng môn thi và điểm TB) và cột nhãn (Xếp loại)
features = data[["Nguvan", "Toan", "Ngoaingu", "Mon 1", "Mon 2", "Mon 3", " TB "]]
labels = data["Xep Loai"]

# Chuyển đổi các cột dữ liệu dạng văn bản thành dạng số (phải thực hiện để huấn luyện mô hình)
features = pd.get_dummies(features)

# Tách dữ liệu thành hai tập Train và Test
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.2, random_state=42)

# Xây dựng mô hình Cây quyết định (Decision Tree)
clf2 = DecisionTreeClassifier(max_depth=10)

# Huấn luyện mô hình trên tập Train
clf2.fit(train_features, train_labels)

# Tạo biểu diễn đồ thị cho cây quyết định
dot_data = export_graphviz(clf2, out_file=None, feature_names=list(train_features.columns),
                           class_names=clf2.classes_, filled=True, rounded=True,
                           special_characters=True)

# Hiển thị đồ thị
graph = graphviz.Source(dot_data)
graph.render(filename='/content/drive/MyDrive/KPDL/TH6/output/Res/dt2', format='png', cleanup=True)
graph.view()

# Xem và nhận xét bộ luật của cây quyết định
tree_rules = export_text(clf2, feature_names=list(features.columns))
print("Decision Tree Rules:")
print(tree_rules)

Decision Tree Rules:
|---  TB  <= 7.37
|   |---  TB  <= 4.99
|   |   |--- class: 0
|   |---  TB  >  4.99
|   |   |---  TB  <= 6.96
|   |   |   |---  TB  <= 6.67
|   |   |   |   |--- Mon 1 <= 1.12
|   |   |   |   |   |--- class: 0
|   |   |   |   |--- Mon 1 >  1.12
|   |   |   |   |   |--- Ngoaingu <= 0.60
|   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |--- Ngoaingu >  0.60
|   |   |   |   |   |   |--- Mon 2 <= 1.25
|   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |--- Mon 2 >  1.25
|   |   |   |   |   |   |   |---  TB  <= 6.62
|   |   |   |   |   |   |   |   |---  TB  <= 6.51
|   |   |   |   |   |   |   |   |   |---  TB  <= 5.01
|   |   |   |   |   |   |   |   |   |   |--- class: TB
|   |   |   |   |   |   |   |   |   |---  TB  >  5.01
|   |   |   |   |   |   |   |   |   |   |--- class: TB
|   |   |   |   |   |   |   |   |---  TB  >  6.51
|   |   |   |   |   |   |   |   |   |--- Ngoaingu <= 5.90
|   |   |   |   |   |   |   |   |   |   |--- class: TB
|   |   

In [ ]:
# Dự đoán kết quả trên tập Test
predictions = clf2.predict(test_features)

# Đánh giá hiệu suất của mô hình trên tập Test
accuracy = accuracy_score(test_labels, predictions)
print("Accuracy on Test Set:", accuracy)

Accuracy on Test Set: 0.9991645781119465


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

# Load the data
df = pd.read_csv("/content/drive/MyDrive/KPDL/TH6/input/2-TNTHPT-XepLoai.csv")

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(pd.get_dummies(df.drop("Xep Loai", axis=1)), df["Xep Loai"], test_size=0.25)

# Create a decision tree classifier
clf = DecisionTreeClassifier()

# Fit the classifier to the training data
clf.fit(X_train, y_train)

# Evaluate the classifier on the test data
y_pred = clf.predict(X_test)

# Print the accuracy of the classifier
print(clf.score(X_test, y_test))

0.9766917293233083
